In [20]:
payload = {
    "datasetName": "random_10",
    "gaParameters": {
        "maxNumberOfIter":300,
        "populationSize": 100,
        "maximumWildCardCount": 0,
        "orientationWeights": [
            1,
            1,
            0.5
        ],
        "geneticAlgorithm": "simpleGa",
        "mate": "normalizedProbabilityVectorSum",
        "mutate": "flipOnePartAtRandom",
        "select": "tournament",
        "objective": "simple",
        "evaluator": "ga",
        "placingHeuristics": "corner",
        "populationDivisionCounts": {
            "elite": 0.2,
            "average": 0.6,
            "worst": 0.2,
            "children": 0.3,
            "mutant": 0.2,
            "winner": 0.2,
            "random": 0.1
        },
        "initialPopulationDivisionCounts": {
            "random": 0.7,
            "greedy": 0.3
        }
    },
    "objectiveParameters": {
        "name": "simple",
        "params": {
            "overlappingPenalizationConstant": 1000,
            "outsideOfAllocatedAreaPenalizationConstant": 0
        }
    }
}

url = "http://localhost:9000/compute/dataset"

headers = {
  'Content-Type': 'application/json'
}

In [2]:
import os
import time
import requests
import json
from pathlib import Path
import os
import shutil
from datetime import datetime
import math


def createDirectory(directory, removeAll=False):
    '''
    Creates a directory if it doesn't exist and removes all files
    from the directory if it already exists and removeAll is True.
    '''
    if os.path.exists(directory) and removeAll:
        shutil.rmtree(directory) # Remove all files in the directory
        print(f'All files in {directory} have been removed.')

    if not os.path.exists(directory):
        os.makedirs(directory) # Create the directory

def checkforFileBlocking(outDir):
    start_time = time.time()  # Get the current time in seconds
    while True:
        if os.path.exists(os.path.join(outDir, "computation_result.json")):
            elapsed_time = time.time() - start_time  # Calculate the elapsed time in seconds
            print(f"{elapsed_time:.2f} sec")
            break
        else:
            time.sleep(10)  # Pause for 5 seconds before checking again

# Population size x best solution

In [ ]:
OUT_DIR_BASE = 'out/cluster_' +  datetime.now().strftime('%Y_%m_%d_%H:%M:%S')

for i in range(10):
    OUT_DIR = f'{OUT_DIR_BASE}/{i+1}'
    print(f'====== RUN {i+1}/{10} ======')
    createDirectory(OUT_DIR)
    for dataset in ['cluster_3_10', 'cluster_4_20']:
        with open(f'/home/slarty/Documents/master-thesis/src/impl/public/datasets/cluster/{dataset}.json', 'r') as file:
            layout = json.load(file)['data']['layout']
        payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = math.sqrt(layout['width']**2 + layout['height']**2) * 4
        payload['datasetName'] = dataset
        size = int(dataset.split('_')[1])
        payload['gaParameters']['populationSize'] = size * 100
        payload['gaParameters']['maxNumberOfIter'] = 1000
        r = requests.request("POST", url, headers=headers, data=json.dumps(payload))
        src = r.json()['outputDirectory'] 
        checkforFileBlocking(src)
        dst = OUT_DIR + '/' + src.split('/')[-1]
        shutil.copytree(src, dst)

print('\nComputation done')

# Cluster

In [ ]:
OUT_DIR_BASE = 'out/cluster_' +  datetime.now().strftime('%Y_%m_%d_%H:%M:%S')

maxRange = 5
# 10.4 - testing maximum wild card count = 1
for i in range(maxRange):
    OUT_DIR = f'{OUT_DIR_BASE}/{i+1}'
    print(f'====== RUN {i+1}/{maxRange} ======')
    createDirectory(OUT_DIR)
    for maximumWildCardCount in [1]: # added 10.4
        for dataset, size in [('cluster_3_10', 30)]:
            print(f'maximumWildCardCount={maximumWildCardCount}, dataset={dataset}')
            with open(f'/home/slarty/Documents/master-thesis/src/impl/public/datasets/cluster/{dataset}.json', 'r') as file:
                layout = json.load(file)['data']['layout']
            payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = math.sqrt(layout['width']**2 + layout['height']**2) * 4
            payload['datasetName'] = dataset
            payload['gaParameters']['populationSize'] = size * 50
            payload['gaParameters']['maxNumberOfIter'] = 300
            payload['gaParameters']['maximumWildCardCount'] = maximumWildCardCount
            r = requests.request("POST", url, headers=headers, data=json.dumps(payload))
            print(r.json())
            src = r.json()['outputDirectory'] 
            checkforFileBlocking(src)
            dst = OUT_DIR + '/' + src.split('/')[-1]
            shutil.copytree(src, dst)

print('\nComputation done')

====== RUN 1/5 ======
maximumWildCardCount=1, dataset=cluster_3_10
{'id': 'cluster_3_10_B1632', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/006_cluster_3_10_B1632'}
2254.25 sec
====== RUN 2/5 ======
maximumWildCardCount=1, dataset=cluster_3_10
{'id': 'cluster_3_10_804CC', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/007_cluster_3_10_804CC'}
2269.27 sec
====== RUN 3/5 ======
maximumWildCardCount=1, dataset=cluster_3_10
{'id': 'cluster_3_10_B39D7', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/008_cluster_3_10_B39D7'}
2278.29 sec
====== RUN 4/5 ======
maximumWildCardCount=1, dataset=cluster_3_10
{'id': 'cluster_3_10_3D655', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/009_cluster_3_10_3D655'}


# Packing

In [ ]:
OUT_DIR_BASE = 'out/packing_' +  datetime.now().strftime('%Y_%m_%d_%H:%M:%S')

for i in range(1):
    OUT_DIR = f'{OUT_DIR_BASE}/{i+1}'
    print(f'====== RUN {i+1}/{10} ======')
    createDirectory(OUT_DIR)
    for dataset in ['packing_10', 'packing_20']:
        with open(f'/home/slarty/Documents/master-thesis/src/impl/public/datasets/packing/{dataset}.json', 'r') as file:
            layout = json.load(file)['data']['layout']
        payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = math.sqrt(layout['width']**2 + layout['height']**2) * 4
        payload['datasetName'] = dataset
        size = int(dataset.split('_')[1])
        payload['gaParameters']['populationSize'] = size * 100
        payload['gaParameters']['maxNumberOfIter'] = 1000
        r = requests.request("POST", url, headers=headers, data=json.dumps(payload))
        src = r.json()['outputDirectory'] 
        checkforFileBlocking(src)
        dst = OUT_DIR + '/' + src.split('/')[-1]
        shutil.copytree(src, dst)

print('\nComputation done')

# Biased sparse cluster

In [ ]:
OUT_DIR_BASE = 'out/biased_sparse_cluster_' +  datetime.now().strftime('%Y_%m_%d_%H:%M:%S')

maxRange = 5
for i in range(maxRange):
    OUT_DIR = f'{OUT_DIR_BASE}/{i+1}'
    print(f'====== RUN {i+1}/{maxRange} ======')
    createDirectory(OUT_DIR)
    for maximumWildCardCount in [0,1]:
        for dataset, size in [('biased_sparse_cluster_2_10', 20), ('biased_sparse_cluster_3_10.json', 30)]:
            print(f'maximumWildCardCount={maximumWildCardCount}, dataset={dataset}')
            with open(f'/home/slarty/Documents/master-thesis/src/impl/public/datasets/biased_sparse_cluster/{dataset}.json', 'r') as file:
                layout = json.load(file)['data']['layout']
            payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = math.sqrt(layout['width']**2 + layout['height']**2) * 4
            payload['datasetName'] = dataset
            payload['gaParameters']['populationSize'] = size * 50
            payload['gaParameters']['maxNumberOfIter'] = 300
            payload['gaParameters']['maximumWildCardCount'] = maximumWildCardCount # added 10.4
            print(r.json())
            r = requests.request("POST", url, headers=headers, data=json.dumps(payload))
            src = r.json()['outputDirectory'] 
            checkforFileBlocking(src)
            dst = OUT_DIR + '/' + src.split('/')[-1]
            shutil.copytree(src, dst)

print('\nComputation done')

# Maximum wild card count vs computation time

In [ ]:
OUT_DIR_BASE = 'out/wild_card_comp_time_' +  datetime.now().strftime('%Y_%m_%d_%H:%M:%S')

for i in range(2, 6):
    OUT_DIR = f'{OUT_DIR_BASE}/{i+1}'
    print(f'====== RUN {i+1}/{10} ======')
    createDirectory(OUT_DIR)
    for dataset in ['random_10']:
        with open(f'/home/slarty/Documents/master-thesis/src/impl/public/datasets/random/{dataset}.json', 'r') as file:
            layout = json.load(file)['data']['layout']
        payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = math.sqrt(layout['width']**2 + layout['height']**2) * 4
        payload['datasetName'] = dataset
        payload['gaParameters']['populationSize'] = 100
        payload['gaParameters']['maxNumberOfIter'] = 300
        
        for maximumWildCardCount in range(0,10):
            print(maximumWildCardCount)
            payload['gaParameters']['maximumWildCardCount'] = maximumWildCardCount
            r = requests.request("POST", url, headers=headers, data=json.dumps(payload))
            src = r.json()['outputDirectory'] 
            checkforFileBlocking(src)
            dst = OUT_DIR + '/' + src.split('/')[-1]
            shutil.copytree(src, dst)

print('\nComputation done')